# Envir Set Up

In [2]:
import random 
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.set_option('display.max_columns', None)

import seaborn as sb


from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_recall_curve,roc_auc_score,fbeta_score,recall_score
from sklearn.metrics import plot_precision_recall_curve,average_precision_score,f1_score
from sklearn.metrics import plot_roc_curve
from sklearn import metrics


# Data preparation

In [3]:
trans = pd.read_csv('transaction_df.csv')


In [4]:
trans.isna().sum()

Unnamed: 0                    0
id                            0
dcs_cust_acct_id              0
pos_busn_dt                   0
mcd_gbal_lcat_id_nu           0
pos_ord_key_id                0
fulfillment_channel           0
pos_tot_net_trn_am            0
pos_tot_tray_itm_qt           0
tot_pnt_earn_cnt_qt      304491
pnt_burn_cnt_qt         5086843
bas_pnt_earn_cnt_qt      307482
bonu_pnt_earn_cnt_qt    4985690
mobl_ord_fl                   0
dtype: int64

In [5]:
# replace the NA with 0
trans = trans.fillna(0)

In [6]:
# rename features
trans = trans.rename({
    'dcs_cust_acct_id': 'customer_id',
    'pos_busn_dt': 'date', # transaction date
    'pos_ord_key_id': 'order_id',
    'mcd_gbal_lcat_id_nu': 'restaurant_id',
    'fulfillment_channel':'channel',
    'pos_tot_net_trn_am': 'transaction_amount', 
    'pos_tot_tray_itm_qt': '#items_purchased',                       
    'tot_pnt_earn_cnt_qt': 'total_points_earned', 
    'pnt_burn_cnt_qt': 'total_points_burned',
    'bas_pnt_earn_cnt_qt': 'base_points_earned', 
    'bonu_pnt_earn_cnt_qt': 'bonus_points_earned',
    'mobl_ord_fl': 'mobile_order'}, 
    axis='columns')

In [7]:
trans['date'] = pd.to_datetime(trans['date'])
trans['day'] = trans.date.dt.day
trans['month'] = trans.date.dt.month
trans['year'] = trans.date.dt.year
trans['weekofday'] = trans.date.dt.dayofweek

In [58]:
feature = ['id','restaurant_id','channel',
           'date','year','month','weekofday','day',
           'transaction_amount','#items_purchased',
          'total_points_earned','total_points_burned',
          'base_points_earned','bonus_points_earned',
              'mobile_order']

In [59]:
df = trans[feature]

In [60]:
df.shape

(5727509, 15)

# Build Transaction feature of Customers

Q1: Who are the churn customers? (what are the metrics for this categorization?)

Q2: Is complaint the reason that leads to the churn of customers? (compare the changes across features before & after complaint)

In [11]:
round(df.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
id,5727509,131849,76583,1,65897,131307,197714,265589
restaurant_id,5727509,195500327431,138580,195500017655,195500248332,195500286747,195500327372,195500893747
year,5727509,2021,0,2021,2021,2021,2021,2022
month,5727509,9,3,1,8,9,11,12
weekofday,5727509,3,2,0,1,3,5,6
day,5727509,15,9,1,8,15,22,31
transaction_amount,5727509,6,6,0,2,5,8,1000
#items_purchased,5727509,4,3,0,2,3,5,888
total_points_earned,5727509,733,773,0,209,500,958,107270
total_points_burned,5727509,-438,1364,-18000,0,0,0,0


# Feature Engineering


## Total Feature:
    - total_freq: buying frequency for each customer
    - sum_transaction_amount: total monetary spend for each customer
    - sum_total_points_earned
    - sum_base_points_earned
    - sum_bonus_points_earned 
    - sum_total_points_burned



In [61]:

# Total buying frequency for each customer
total_freq = df.groupby('id')['id'].count().to_frame().rename({'id':'total_freq'},axis='columns')
df = pd.merge(df,total_freq, 
                 how = 'left',
                 on=['id'])
feature = feature+['total_freq']

In [62]:
# Total money spent, points earned and burn for each customer
total_monetary = df.groupby('id')['transaction_amount',
                                    'total_points_earned',
                                    'base_points_earned', 
                                    'bonus_points_earned', 
                                    'total_points_burned'].sum()
monetary_feature = ['sum_transaction_amount',
                          'sum_total_points_earned',
                          'sum_base_points_earned', 
                          'sum_bonus_points_earned', 
                          'sum_total_points_burned']
total_monetary.columns = monetary_feature

feature = feature + monetary_feature
df = pd.merge(df,total_monetary, 
                 how = 'left',
                 on=['id'])


## Daily Feature
    - describe daily fequency,transaction amount, item purchased, bonus points earned/burned for each customer


In [63]:
# Daily monetary spent, #item purchased, points earned and burn for each customer
daily_sum = trans.groupby(['id', 'date'])['id',
                                               "transaction_amount",
                                               "#items_purchased", 
                                               "total_points_earned",
                                               "total_points_burned", 
                                               "base_points_earned", 
                                               "bonus_points_earned"].sum()


In [64]:
daily_feature = ['daily_freq',"daily_transaction_amount",
                 "daily_#items_purchased",
                 "daily_total_points_earned",
                 "daily_total_points_burned", 
                 "daily_base_points_earned", 
                 "daily_bonus_points_earned"]

daily_sum.columns = daily_feature

feature = feature + daily_feature

In [65]:

daily_df = daily_sum.reset_index()

daily_df['days_since_last_purchase'] = daily_df[["id", "date"]].groupby('id')['date'].apply(lambda x: (x - x.shift(1)).dt.days)


In [68]:
feature = feature + ['days_since_last_purchase']

In [67]:
df = pd.merge(df,daily_df, 
                 how = 'left',
                 on=['id','date'])

## Cumulative Feature
    - describe the cumulative sum of daily feature regarding item puchased, transaction amoung, bonus point

In [71]:
daily_cumsum = daily_sum.groupby(level=0).cumsum()


In [73]:
cumsum_feature = ['cumsum_freq',
                  "cumsum_transaction_amount",
                  "cumsum_#items_purchased", 
                  "cumsum_total_points_earned",
                  "cumsum_total_points_burned", 
                  "cumsum_base_points_earned", 
                  "cumsum_bonus_points_earned"]

In [74]:
daily_cumsum.columns = cumsum_feature

In [75]:
feature = feature + cumsum_feature

In [76]:
daily_cumsum_df = daily_cumsum.reset_index()

In [77]:
df = pd.merge(df,daily_cumsum_df, how='left',on=['id','date'])


In [3]:
# df = pd.read_csv('transaction data with FRM features.csv')

In [5]:
df['Drive_Thru'] = df['channel'].apply(lambda x: 1 if x == 'Drive Thru' else 0)
df['Front_Counter'] = df['channel'].apply(lambda x: 1 if x == 'Front Counter' else 0)
df['Curbside'] = df['channel'].apply(lambda x: 1 if x == 'Curbside' else 0)

In [8]:
df.head()

,Unnamed: 0,id,restaurant_id,channel,date,year,month,weekofday,day,transaction_amount,#items_purchased,total_points_earned,total_points_burned,base_points_earned,bonus_points_earned,mobile_order,total_freq,sum_transaction_amount,sum_total_points_earned,sum_base_points_earned,sum_bonus_points_earned,sum_total_points_burned,daily_freq,daily_transaction_amount,daily_#items_purchased,daily_total_points_earned,daily_total_points_burned,daily_base_points_earned,daily_bonus_points_earned,days_since_last_purchase,cumsum_freq,cumsum_transaction_amount,cumsum_#items_purchased,cumsum_total_points_earned,cumsum_total_points_burned,cumsum_base_points_earned,cumsum_bonus_points_earned,Drive_Thru,Front_Counter,Curbside
0,0,21879,195500337555,Drive Thru,2021-07-01,2021,7,3,1,24,11,2352,0,2352,0,0,115,1752,175736,174179,1557,-223500,21879,24,11,2352,0,2352,0,NaN,21879,24,11,2352,0,2352,0,1,0,0
1,1,69320,195500296326,Front Counter,2021-07-01,2021,7,3,1,0,1,0,0,0,0,0,198,367,36757,34983,1774,-36000,69320,0,1,0,0,0,0,NaN,69320,0,1,0,0,0,0,0,1,0
2,2,198919,195500286556,Drive Thru,2021-07-01,2021,7,3,1,12,7,1159,0,1159,0,1,28,249,24857,24857,0,-18000,198919,12,7,1159,0,1159,0,NaN,198919,12,7,1159,0,1159,0,1,0,0
3,3,200321,195500321196,Drive Thru,2021-07-01,2021,7,3,1,6,4,618,0,618,0,0,176,618,64575,61848,2727,-48000,400642,8,6,818,0,818,0,NaN,400642,8,6,818,0,818,0,1,0,0
4,4,121937,195500337555,Front Counter,2021-07-01,2021,7,3,1,0,1,0,-1500,0,0,1,9,155,15469,15469,0,-16500,121937,0,1,0,-1500,0,0,NaN,121937,0,1,0,-1500,0,0,0,1,0


In [9]:
df.to_csv("Engineered Transaction Data.csv")